# Task description
- Classify the speakers of given features.
- Main goal: Learn how to use transformer.
- Baselines:
  - Easy: Run sample code and know how to use transformer.
  - Medium: Know how to adjust parameters of transformer.
  - Hard: Construct [conformer](https://arxiv.org/abs/2005.08100) which is a variety of transformer. 

- Other links
  - Kaggle: [link](https://www.kaggle.com/t/859c9ca9ede14fdea841be627c412322)
  - Slide: [link](https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW04/HW04.pdf)
  - Data: [link](https://drive.google.com/file/d/1T0RPnu-Sg5eIPwQPfYysipfcz81MnsYe/view?usp=sharing)
  - Video (Chinese): [link](https://www.youtube.com/watch?v=EPerg2UnGaI)
  - Video (English): [link](https://www.youtube.com/watch?v=Gpz6AUvCak0)
  - Solution for downloading dataset fail.: [link](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e?usp=sharing)

# Download dataset
- Please follow [here](https://drive.google.com/drive/folders/13T0Pa_WGgQxNkqZk781qhc5T9-zfh19e?usp=sharing) to download data
- Data is [here](https://drive.google.com/file/d/1FCjdQkMk9YAIhRkUjbbbv89YO9HhPD8J/view?usp=sharing)

In [1]:
!gdown '1EYKvIlR4PwkpKqu2bibw2FhmCEgW-_iC' --output Dataset.zip
!unzip Dataset.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: Dataset/uttr-e781829441ab4b489bc73a8b13f15961.pt  
  inflating: __MACOSX/Dataset/._uttr-e781829441ab4b489bc73a8b13f15961.pt  
  inflating: Dataset/uttr-68c399430bda4d7da059e9d3d9f77503.pt  
  inflating: __MACOSX/Dataset/._uttr-68c399430bda4d7da059e9d3d9f77503.pt  
  inflating: Dataset/uttr-6531942fcdc84d67afb1d3c73aca8a87.pt  
  inflating: __MACOSX/Dataset/._uttr-6531942fcdc84d67afb1d3c73aca8a87.pt  
  inflating: Dataset/uttr-b17319bdc2864b2b81d1ff0f90bcadc2.pt  
  inflating: __MACOSX/Dataset/._uttr-b17319bdc2864b2b81d1ff0f90bcadc2.pt  
  inflating: Dataset/uttr-d3b4e141417e48bfa674ae1008420e43.pt  
  inflating: __MACOSX/Dataset/._uttr-d3b4e141417e48bfa674ae1008420e43.pt  
  inflating: Dataset/uttr-34f6d27303f64a178d3ca59067b17750.pt  
  inflating: __MACOSX/Dataset/._uttr-34f6d27303f64a178d3ca59067b17750.pt  
  inflating: Dataset/uttr-e730eb663d524370a328a7a0e6cfeead.pt  
  inflating: __MACOSX/Dataset/._uttr-e730eb663d524370a328a7a0e6cfeead.pt  
  infl

# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len
 
    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]
 
    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]
 
    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
  def __len__(self):
    return len(self.data)
 
  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))
 
    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, len(mel), speaker
 
  def get_speaker_number(self):
    return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [3]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, len, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.IntTensor(len), torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.models as m


class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.2):
    super().__init__()
    self.prenet = nn.Linear(40, d_model)
    self.conformer = m.Conformer(
        input_dim = d_model,
        num_heads = 4,
        ffn_dim = 144,
        num_layers = 16,
        depthwise_conv_kernel_size = 3,
        dropout = dropout,
    )
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels, lens):
    outs = self.prenet(mels)
    outs, _ = self.conformer(outs, lens)
    stats = outs.mean(dim=1)
    outs = self.pred_layer(stats)
    return outs


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [7]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


# Model Function
- Model forward function.

In [8]:
class AMSoftmaxLoss(nn.Module):

    def __init__(self, s=30.0, m=0.4):
        super(AMSoftmaxLoss, self).__init__()
        self.s = s
        self.m = m

    def forward(self, x, labels):
        assert len(x) == len(labels)
        numerator = self.s * (torch.diagonal(x.transpose(0, 1)[labels]) - self.m)
        excl = torch.cat([torch.cat((x[i, :y], x[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)
        return -torch.mean(L)

In [9]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, lens, labels = batch
  mels = mels.to(device)
  lens = lens.to(device)
  labels = labels.to(device)

  outs = model(mels, lens)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


# Validate
- Calculate accuracy of the validation set.

In [10]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


In [11]:
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# Main function

In [12]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 300000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  same_seeds(0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  opt_model = torch.compile(model).to(device)
  criterion = nn.CrossEntropyLoss()
  #criterion = AMSoftmaxLoss(speaker_num, speaker_num, s=30.0, m=0.4).to(device)
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, opt_model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())


Train: 100% 2000/2000 [17:25<00:00,  1.91 step/s, accuracy=0.03, loss=4.75, step=2000]
Valid: 100% 6336/6344 [00:15<00:00, 402.73 uttr/s, accuracy=0.10, loss=4.56]
Train: 100% 2000/2000 [03:44<00:00,  8.92 step/s, accuracy=0.16, loss=4.15, step=4000]
Valid: 100% 6336/6344 [00:15<00:00, 420.69 uttr/s, accuracy=0.18, loss=3.93]
Train: 100% 2000/2000 [03:49<00:00,  8.73 step/s, accuracy=0.16, loss=4.11, step=6000]
Valid: 100% 6336/6344 [00:15<00:00, 421.51 uttr/s, accuracy=0.23, loss=3.58]
Train: 100% 2000/2000 [03:48<00:00,  8.74 step/s, accuracy=0.34, loss=3.09, step=8000]
Valid: 100% 6336/6344 [00:16<00:00, 391.94 uttr/s, accuracy=0.32, loss=3.12]
Train: 100% 2000/2000 [03:46<00:00,  8.83 step/s, accuracy=0.47, loss=2.60, step=1e+4]
Valid: 100% 6336/6344 [00:20<00:00, 305.69 uttr/s, accuracy=0.40, loss=2.59]
Train:   0% 0/2000 [00:00<?, ? step/s]

Step 10000, best model saved. (accuracy=0.3995)


Train: 100% 2000/2000 [03:46<00:00,  8.83 step/s, accuracy=0.47, loss=2.08, step=12000]
Valid: 100% 6336/6344 [00:15<00:00, 415.70 uttr/s, accuracy=0.47, loss=2.26]
Train: 100% 2000/2000 [03:49<00:00,  8.70 step/s, accuracy=0.56, loss=2.00, step=14000]
Valid: 100% 6336/6344 [00:15<00:00, 416.52 uttr/s, accuracy=0.50, loss=2.10]
Train: 100% 2000/2000 [03:48<00:00,  8.74 step/s, accuracy=0.47, loss=2.09, step=16000]
Valid: 100% 6336/6344 [00:15<00:00, 407.60 uttr/s, accuracy=0.54, loss=1.92]
Train: 100% 2000/2000 [03:49<00:00,  8.72 step/s, accuracy=0.56, loss=1.72, step=18000]
Valid: 100% 6336/6344 [00:15<00:00, 419.98 uttr/s, accuracy=0.58, loss=1.76]
Train: 100% 2000/2000 [03:54<00:00,  8.54 step/s, accuracy=0.69, loss=1.25, step=2e+4]
Valid: 100% 6336/6344 [00:15<00:00, 404.22 uttr/s, accuracy=0.58, loss=1.78]
Train:   0% 2/2000 [00:00<06:02,  5.52 step/s, accuracy=0.56, loss=1.79, step=2e+4]

Step 20000, best model saved. (accuracy=0.5783)


Train: 100% 2000/2000 [03:48<00:00,  8.74 step/s, accuracy=0.69, loss=1.46, step=22000]
Valid: 100% 6336/6344 [00:16<00:00, 380.04 uttr/s, accuracy=0.64, loss=1.50]
Train: 100% 2000/2000 [03:48<00:00,  8.77 step/s, accuracy=0.81, loss=1.14, step=24000]
Valid: 100% 6336/6344 [00:16<00:00, 390.96 uttr/s, accuracy=0.66, loss=1.40]
Train: 100% 2000/2000 [03:47<00:00,  8.78 step/s, accuracy=0.59, loss=1.19, step=26000]
Valid: 100% 6336/6344 [00:16<00:00, 392.61 uttr/s, accuracy=0.68, loss=1.35]
Train: 100% 2000/2000 [03:50<00:00,  8.68 step/s, accuracy=0.84, loss=0.87, step=28000]
Valid: 100% 6336/6344 [00:15<00:00, 402.90 uttr/s, accuracy=0.67, loss=1.34]
Train: 100% 2000/2000 [03:50<00:00,  8.67 step/s, accuracy=0.66, loss=1.19, step=3e+4]
Valid: 100% 6336/6344 [00:16<00:00, 392.82 uttr/s, accuracy=0.69, loss=1.29]
Train:   0% 2/2000 [00:00<06:12,  5.36 step/s, accuracy=0.56, loss=1.72, step=3e+4]

...

Step 280000, best model saved. (accuracy=0.9329)


Train: 100% 2000/2000 [04:00<00:00,  8.30 step/s, accuracy=1.00, loss=0.03, step=282000]
Valid: 100% 6336/6344 [00:16<00:00, 395.28 uttr/s, accuracy=0.93, loss=0.30]
Train: 100% 2000/2000 [03:49<00:00,  8.71 step/s, accuracy=1.00, loss=0.04, step=284000]
Valid: 100% 6336/6344 [00:15<00:00, 398.58 uttr/s, accuracy=0.93, loss=0.30]
Train: 100% 2000/2000 [03:48<00:00,  8.74 step/s, accuracy=1.00, loss=0.02, step=286000]
Valid: 100% 6336/6344 [00:15<00:00, 400.90 uttr/s, accuracy=0.93, loss=0.31]
Train: 100% 2000/2000 [03:49<00:00,  8.70 step/s, accuracy=0.97, loss=0.05, step=288000]
Valid: 100% 6336/6344 [00:16<00:00, 381.74 uttr/s, accuracy=0.93, loss=0.32]
Train: 100% 2000/2000 [03:54<00:00,  8.53 step/s, accuracy=1.00, loss=0.01, step=290000]
Valid: 100% 6336/6344 [00:16<00:00, 385.61 uttr/s, accuracy=0.93, loss=0.33]
Train:   0% 1/2000 [00:00<06:50,  4.87 step/s, accuracy=1.00, loss=0.01, step=290001]

Step 290000, best model saved. (accuracy=0.9329)


Train: 100% 2000/2000 [03:55<00:00,  8.49 step/s, accuracy=1.00, loss=0.01, step=292000]
Valid: 100% 6336/6344 [00:15<00:00, 399.07 uttr/s, accuracy=0.93, loss=0.34]
Train: 100% 2000/2000 [03:56<00:00,  8.46 step/s, accuracy=1.00, loss=0.01, step=294000]
Valid: 100% 6336/6344 [00:15<00:00, 405.31 uttr/s, accuracy=0.93, loss=0.32]
Train: 100% 2000/2000 [04:01<00:00,  8.28 step/s, accuracy=1.00, loss=0.01, step=296000]
Valid: 100% 6336/6344 [00:16<00:00, 384.55 uttr/s, accuracy=0.93, loss=0.33]
Train: 100% 2000/2000 [04:00<00:00,  8.32 step/s, accuracy=1.00, loss=0.02, step=298000]
Valid: 100% 6336/6344 [00:15<00:00, 414.52 uttr/s, accuracy=0.93, loss=0.33]
Train: 100% 2000/2000 [04:04<00:00,  8.16 step/s, accuracy=0.97, loss=0.08, step=3e+5]
Valid: 100% 6336/6344 [00:16<00:00, 393.64 uttr/s, accuracy=0.92, loss=0.32]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 300000, best model saved. (accuracy=0.9329)


# Inference

## Dataset of inference

In [13]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel, len(mel)


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels, lens = zip(*batch)

  return feat_paths, torch.stack(mels), torch.IntTensor(lens)


## Main funcrion of Inference

In [14]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "model_path": "./model.ckpt",
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_path,
  output_path,
):
  same_seeds(0)
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=8,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)
  model.load_state_dict(torch.load(model_path))
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels, lens in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      lens = lens.to(device)
      outs = model(mels, lens)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/6000 [00:00<?, ?it/s]

## Automatic downloads

In [15]:
from google.colab import files
!zip -r 'output.zip' 'output.csv'
!zip -r 'model.zip' 'model.ckpt'
files.download('output.zip')
files.download('model.zip')

  adding: output.csv (deflated 54%)
  adding: model.ckpt (deflated 9%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Termination

In [ ]:
import time
from google.colab import runtime
time.sleep(30)
runtime.unassign()